In [21]:
import pandas as pd
import numpy as np
from sktime.performance_metrics.forecasting import mean_absolute_error

### Import data - calculate adjustment directions

In [111]:
# import datasets
test_data = pd.read_csv("../../Outputs/Forecasts/Test_h18.csv")
original_forecasts = pd.read_csv("../../Outputs/Forecasts/TES_h18_original.csv")
protected_forecasts = pd.read_csv("../../Outputs/Forecasts/TES_h18_Top_0.4.csv")

In [94]:
# collapsed_original = pd.concat([row for i, row in original_forecasts.iterrows()], ignore_index=True)
# collapsed_protected = pd.concat([row for i, row in protected_forecasts.iterrows()], ignore_index=True)
# collapsed_test = pd.concat([row for i, row in test_data.iterrows()], ignore_index=True)

In [95]:
# split into lists
test_data = [test_data[col] for col in test_data]
original_forecasts = [original_forecasts[col] for col in original_forecasts]
protected_forecasts = [protected_forecasts[col] for col in protected_forecasts]

In [96]:
# booleans for adjustment direction
adjusted_up = [protected_forecasts[i] > original_forecasts[i] for i, _ in enumerate(protected_forecasts)]
adjusted_down = [protected_forecasts[i] < original_forecasts[i] for i, _ in enumerate(protected_forecasts)]

***

### Function to Calculate AvgRelMAE

In [97]:
def AvgRelMAE(test_series, original_fcasts, protected_fcasts, t):
    r = [mean_absolute_error(test_series[i], protected_fcasts[i])/mean_absolute_error(test_series[i], original_fcasts[i]) for i, _ in enumerate(test_series)]
    n = [len(j) for i, j in enumerate(protected_fcasts)]
    nlogr = [n[i]*np.log(r[i]) for i, _ in enumerate(r)]
    # nlogr.sort()
    # trim_val = int(t*len(nlogr))
    # nlogr = nlogr[trim_val:]
    # nlogr = nlogr[:-trim_val]
    return (1 - np.exp((1/np.sum(n))*np.sum(nlogr))) * 100

### Calculate AvgRelMAE for Positive Adjusted Forecasts

In [98]:
test_up = [test_data[i][j] for i, j in enumerate(adjusted_up) if len(test_data[i][j]) > 0]

In [99]:
orig_up = [original_forecasts[i][j] for i, j in enumerate(adjusted_up) if len(original_forecasts[i][j]) > 0]

In [100]:
protected_up = [protected_forecasts[i][j] for i, j in enumerate(adjusted_up) if len(protected_forecasts[i][j]) > 0]

In [101]:
AvgRelMAE(test_up, orig_up, protected_up, 0.05)

12.528727872857681

***

### Calculate AvgRelMAE for Negative Adjusted Forecasts

In [102]:
test_down = [test_data[i][j] for i, j in enumerate(adjusted_down) if len(test_data[i][j]) > 0]

In [103]:
orig_down = [original_forecasts[i][j] for i, j in enumerate(adjusted_down) if len(original_forecasts[i][j]) > 0]

In [104]:
protected_down = [protected_forecasts[i][j] for i, j in enumerate(adjusted_down) if len(protected_forecasts[i][j]) > 0]

In [105]:
AvgRelMAE(test_down, orig_down, protected_down, 0.05)

11.868049685914862

***

### Proportion of Forecasts Adjusted in Either Direction

In [106]:
np.mean(adjusted_up)

0.4098687294889827

In [107]:
np.mean(adjusted_down)

0.5901312705110173

### Magnitude of Adjustments

In [108]:
absolute_size_up = [np.abs(orig_up[i] - protected_up[i]) for i, _ in enumerate(orig_up)]

In [109]:
np.mean(np.concatenate(absolute_size_up))

207.58056656268369

In [110]:
np.mean(absolute_size_up)

C:\Users\Cameron\anaconda3\lib\site-packages\numpy\core\_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan])

In [43]:
absolute_size_down = [np.mean(np.abs(orig_down[i] - protected_down[i])) for i, _ in enumerate(orig_down)]

In [44]:
np.mean(absolute_size_down)

234.18409822992004